## Importsetup

In [2]:
###########################################################################
from app.main.extraction.dxf_extraction_coordinator import DxfExtractionCoordinator
from app.main.models.wall import Wall 
from app.main.models.architecture_wall import Architecture_Wall
from app.main.models.point import Point 
from app.main.models.pillar import Pillar
from app.main.services.walls_extraction import WallsExtraction
from app.main.services.dxf_extraction_simple import DxfExtractionSimple
from app.main.dtos.request.extraction_request_file import ExtractionRequestFile
from app.main.models.line import Line 
###########################################################################
from shapely.geometry.polygon import LinearRing
from shapely.geometry.polygon import Polygon as shapePolygon
from shapely.geometry import Point as shapePoint
from shapely.geometry import LineString
import math
import numpy as np
import itertools as it
from scipy.spatial.distance import cdist
import ezdxf
import uuid

import matplotlib.pyplot as plt
import operator

###########################################################################
file_path_simple = '../../media/Einfache_Plaene_Compass/'
file_path_architecture = '../../media/Architektenplaene/'
###########################################################################
file_name_simple = 'R-Bau_D_EG.dxf'
file_name_architecutre = 'R-Bau_A_EG.dxf'
###########################################################################
simple_dxf = ezdxf.readfile(file_path_simple+file_name_simple)
architecture_dxf = ezdxf.readfile(file_path_architecture+file_name_architecutre)
simple_model = simple_dxf.modelspace()
arichtecture_model = architecture_dxf.modelspace()

In [4]:
'''This cell gets all the circle entities from the architecture plan.'''
pillars = arichtecture_model.query("CIRCLE[layer=='A_01_STUETZE']")

### In the block below there will be created a dxf-File for analysis purposes.

In [6]:
doc = ezdxf.new('R2000')
msp = doc.modelspace()
#Read pillar circles from architecture wall
architecture_polylines = []
architecture_lines = []
saeulen_polylines = arichtecture_model.query("LWPolyline[layer=='A_01_STUETZE']")#.first.get_points('xy')
saeulen_lines = arichtecture_model.query("LINE[layer=='A_01_STUETZE']")

pillars = arichtecture_model.query("CIRCLE[layer=='A_01_STUETZE']")
doc.layers.new(name='SAULEN_LINES', dxfattribs={'linetype': 'DASHED', 'color': 1})
doc.layers.new(name='SAULEN_POLYLINES', dxfattribs={'linetype': 'DASHED', 'color': 2})
doc.layers.new(name='SAULEN_CIRCLES', dxfattribs={'linetype': 'DASHED', 'color': 3})

print(len(saeulen_lines))
print(len(saeulen_polylines))
for polyline in saeulen_polylines:
    architecture_polylines.append(polyline)

for line in saeulen_lines:
    architecture_lines.append(line)


for x in architecture_polylines:
    msp.add_lwpolyline(x, dxfattribs={'layer': 'SAULEN_POLYLINES'})

#add lines to drawing
coordinates = []
for e in architecture_lines:
    if e.dxftype() == 'LINE':
        tupel = (e.dxf.start, e.dxf.end)
        coordinates.append(tupel)

for x in coordinates:
    msp.add_line(x[0], x[1], dxfattribs={'layer': 'SAULEN_LINES'})

for x in pillars:
    msp.add_circle(x.dxf.center, x.dxf.radius, dxfattribs={'layer': 'SAULEN_CIRCLES'})

doc.saveas("pillars.dxf")

1
19


In [7]:
''' This method creates an architecture plan pillar with a center coordinate point an a radius.'''
def create_architecture_pillar(center: tuple, radius: float):
    center_point = Point(center[0], center[1])
    pillar_id = str(uuid.uuid4())
    return Pillar(center_point=center_point, radius=radius, pillar_id=pillar_id)

### The following block transform all pillars intow achritecutre pillars.

In [9]:
#transform pillars to model
architecture_pillars = []
for pillar in pillars:
    architecture_pillar = create_architecture_pillar(pillar.dxf.center, pillar.dxf.radius)
    architecture_pillars.append(architecture_pillar)

### The following block extracts all polygons from the given simple plan.

In [10]:
#read simple plan
dxf_file = ezdxf.readfile(file_path_simple + file_name_simple)
dxf_extraction_simple = DxfExtractionSimple()
simple_polygons, polygons_with_virtual_entities, extracted_functionalities, counter_polygon_closed = dxf_extraction_simple.get_all_polygons(dxf_file ,file_path_simple + file_name_simple)

2021-06-18 10:42:30.423 | SUCCESS  | app.main.services.orientation_extraction:get_north_orientation:116 - Compass extraction successful, return coordinate system
2021-06-18 10:42:30.423 | INFO     | app.main.services.dxf_extraction_simple:get_all_polygons:97 - Orientation will be calculated with compass.
2021-06-18 10:42:30.424 | SUCCESS  | app.main.services.extraction_helper:get_modelspace:191 - sucessfully get_modelspace() from dxf_file: <ezdxf.document.Drawing object at 0x000001AD826C6190>
2021-06-18 10:42:30.425 | DEBUG    | app.main.services.extraction_helper:get_outline_polygon:200 - get_outline_polygon for modelspace: <ezdxf.layouts.layout.Modelspace object at 0x000001AD8507B1F0>
2021-06-18 10:42:30.475 | DEBUG    | app.main.services.dxf_extraction_simple:get_all_polygons:145 - got all room_polylines on layer: RAUMPOLYGON
2021-06-18 10:42:30.824 | INFO     | app.main.services.dxf_extraction_simple:get_all_polygons:192 - Extracting neighbours...
Neighbours - Progress: 100%|██████

In [7]:
''' This method calculates the lenght in between two points'''
def get_length_between_points(point_1: Point, point_2: Point) -> float:
    line_string = LineString([(point_1.x_coordinate, point_1.y_coordinate), (point_2.x_coordinate, point_2.y_coordinate)])
    return line_string.length

In [11]:
''' This method maps all pillars to simple pillars extracted from the simple plan'''
def map_pillars_to_simple_pillars(simple_polygons: list, architecture_pillars: list[Pillar]):
    #loop trough simple polygons
    min_pillar_distances = []
    threshold = 0.00104
    for polygon in simple_polygons:
        #if there are pillar in the polygon
        if polygon.pillars:
            #loop through all simple pillars
            for simple_pillar in polygon.pillars:
                pillar_distances = []
                potential_pillar = None
                potential_pillar_distance = 0
                # for each architecture pillar there will be calculated the distance to the simple pillar
                for architecture_pillar in architecture_pillars:
                    # calculate the distance between simple pillar and architecutre pillar
                    pillar_distance = get_length_between_points(simple_pillar.start_point, architecture_pillar.center_point)
                    if not potential_pillar:
                        # if potentail pillar is not none
                        # overwrite with value of architecture pillar and pillar distance
                        potential_pillar = architecture_pillar
                        potential_pillar_distance = pillar_distance
                    elif pillar_distance < potential_pillar_distance:
                        # else check if the distance is smaller than the potential distance
                        potential_pillar = architecture_pillar
                        potential_pillar_distance = pillar_distance
                
                # Check if potential pillar distance is smaller than the range of the radius plus the threshold value
                if potential_pillar_distance < potential_pillar.radius + threshold:
                    print("Pillar with id {} matched!".format(potential_pillar.pillar_id))
                    simple_pillar.pillar_id = potential_pillar.pillar_id
                    simple_pillar.center_point = potential_pillar.center_point
                    simple_pillar.radius = potential_pillar.radius
                else:
                    print("false")
                    print("Distanz: {}, radius: {}.".format(str(potential_pillar_distance), str(potential_pillar.radius)))
                    print("Differenz: {}".format(str(potential_pillar_distance - potential_pillar.radius)))
    
    # @nico bitte was?
    pillars_flat = []
    for polygon in simple_polygons:
        if polygon.pillars:
            for simple_pillar in polygon.pillars:
                pillars_flat.append(simple_pillar)
                print(simple_pillar)
    
    pillar_ids = []
    for pillar in pillars_flat:
        pillar_ids.append(pillar.pillar_id)
    print("Amount of pillars: {}".format(str(len(pillar_ids))))
    has_duplicate_items = any(pillar_ids.count(x) > 1 for x in pillar_ids)
    print("Duplicate pilllar matches? {}".format(str(has_duplicate_items)))


In [22]:
# Testing the method
map_pillars_to_simple_pillars(simple_polygons, architecture_pillars)

Pillar with id d114a0d4-bc17-4c64-ba91-c014c8817631 matched!
Pillar with id 8f65d8bc-4847-4adb-9e3d-e026a5e8d18f matched!
Pillar with id bdddd078-397b-46d9-adc0-71a4a333f3b1 matched!
Pillar with id c561f4bc-a0de-4f15-8f73-25114bb23acc matched!
Pillar with id 616bc19c-0336-480b-bcb3-b9297bf4a8cf matched!
Pillar with id 1189436c-1eb0-421c-8631-33200c6750f6 matched!
Pillar with id 4b160b21-7eb0-4865-a7be-ab2aaa44c64c matched!
Pillar with id 4e229b0b-be50-4f5b-b743-992c554c54f1 matched!
Pillar with id b41fbcf6-ef26-43fe-aa29-9e993e489967 matched!
Pillar with id c1ffb4a9-1b02-4ba5-b941-26ac4d69e98e matched!
Pillar with id 511f5ea3-95fe-403c-adb3-fb3062eb850f matched!
Pillar with id 214da17f-cfa9-4083-ad0f-3f47c9513a27 matched!
Pillar with id 1f761177-4812-4173-88a9-807d942e26aa matched!
Pillar with id def2bde3-9950-42f2-972d-260135cb9309 matched!
Pillar with id 77897350-6a6c-4bc3-b471-f56b2b6bda82 matched!
Pillar with id 410f0d32-e61f-4e52-b363-2a795cc169e8 matched!
Pillar with id c5648b7a-